### Практика

#### Скачивание данных

Работа с google drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#копирование cancer_detection.zip в content
!cp /content/drive/MyDrive/Colab_Notebooks/diploma/dataset/cancer_detection.zip /content/

In [ ]:
#проверка, что cancer_detection.zip находится в content
!ls /content/

cancer_detection.zip  drive  sample_data


In [ ]:
!unzip -qq "/content/cancer_detection.zip"

In [ ]:
import numpy as np
import timeit
import matplotlib.pyplot as plt

from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms

from sklearn.metrics import roc_auc_score

In [ ]:
resnet_transforms = transforms.Compose([
        transforms.Resize(256), # размер каждой картинки будет приведен к 256*256
        transforms.CenterCrop(224), # у картинки будет вырезан центральный кусок размера 224*224
        # transforms.RandomPerspective(distortion_scale=0.6, p=1.0),
        transforms.ToTensor(), # картинка из питоновского массива переводится в формат torch.Tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # значения пикселей картинки нормализуются
    ])

In [ ]:
train_data = datasets.ImageFolder('./cancer_detection/train', transform=resnet_transforms)
#val_data = datasets.ImageFolder('./cancer_detection/valid', transform=resnet_transforms)
test_data = datasets.ImageFolder('./cancer_detection/test', transform=resnet_transforms)

# делим тренировочную часть на train и val

# в тренировочную выборку отнесем 80% всех картинок
train_size = int(len(train_data) * 0.8)
# в валидационную — остальные 20%
val_size = len(train_data) - train_size

train_data, val_data = torch.utils.data.random_split(train_data, [train_size, val_size])

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

#### Замена последнего слоя сети

Загрузим модель, которую будем дообучать:

In [ ]:
model = models.resnet18(pretrained=True)
model

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 345MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

Заменим последний слой сети на новый, содержащий 70 нейронов (так как у нас 70 классов в датасете):

In [ ]:
model.fc = nn.Linear(512, 1)
# скорее всего сюда добавим лосс ф-уию либо туда где она вызывается
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

#### Заморозка слоев

In [ ]:
# model.children() выдает список сабмодулей нейросети
# в нашем случае это блоки resnet 
len(list(model.children()))

10

Заморозим все сверточные слои:

In [ ]:
# проходимся по блокам нейросети
for i, layer in enumerate(model.children()):

  # заморозим первые девять блоков 
  if i < 9:
    # проходимся по всем весам (параметрам) блока
    for param in layer.parameters():
      # замораживаем паарметр
      param.requires_grad = False

Соберем весь код подготовки вместе:

In [ ]:
def create_model(model, num_out_classes, num_freeze_layers=None):
    # замена последнего слоя сети
    model.fc = nn.Linear(512, num_out_classes)

    # заморозка слоев
    if num_freeze_layers is not None:
        for i, layer in enumerate(model.children()):
            if i < num_freeze_layers:
                for param in layer.parameters():
                    param.requires_grad = False
    
    return model

In [ ]:
model = create_model(models.resnet18(pretrained=True), 1)

#### Обучение сети

Перенесем нашу нейросеть на GPU, если GPU доступен:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

Напишем функции для обучения сети

In [ ]:
from IPython.display import clear_output

def evaluate(model, dataloader, loss_fn):
    losses = []
    predictions = []
    targets = []

    num_correct = 0
    num_elements = 0

    for i, batch in enumerate(dataloader):
        # получаем текущий батч
        X_batch, y_batch = batch
        num_elements += len(y_batch)

        # добавляем дополнительное измерение к целевым меткам
        y_batch = y_batch.unsqueeze(1).float().to(device)

        # эта строка запрещает вычисление градиентов
        with torch.no_grad():
            # получаем ответы сети на картинки батча
            logits = model(X_batch.to(device))

            # вычисляем лосс на текущем батче
            loss = loss_fn(logits, y_batch)
            losses.append(loss.item())

            # вычисляем ответы сети
            y_pred = torch.round(torch.sigmoid(logits)) # применяем сигмоиду и округляем

            # вычисляем количество правильных ответов сети в текущем батче
            num_correct += torch.sum(y_pred.cpu() == y_batch.cpu())

            # сохраняем прогнозы и целевые метки для вычисления ROC AUC
            predictions.append(y_pred.cpu().numpy())
            targets.append(y_batch.cpu().numpy())

    # вычисляем итоговую долю правильных ответов
    accuracy = (num_correct / num_elements).cpu().numpy()

    # вычисляем ROC AUC
    predictions = np.concatenate(predictions)
    targets = np.concatenate(targets)
    roc_auc = roc_auc_score(targets, predictions)

    return accuracy, np.mean(losses), roc_auc



import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from IPython.display import clear_output

def train(model, loss_fn, optimizer, n_epoch=3):
    train_losses = []
    train_accuracies = []
    train_roc_aucs = []
    val_losses = []
    val_accuracies = []
    val_roc_aucs = []
    train_iterations = []
    
    # цикл обучения сети
    for epoch in range(n_epoch):
        print("Epoch:", epoch+1)
        model.train(True)
        
        for i, batch in enumerate(train_loader):
            # получаем текущий батч
            X_batch, y_batch = batch 
            y_batch = y_batch.unsqueeze(1).float().to(device)
            
            # forward pass (получение ответов на батч картинок)
            logits = model(X_batch.to(device)) 
            
            # вычисление лосса от выданных сетью ответов и правильных ответов на батч
            loss = loss_fn(logits, y_batch) 
            train_losses.append(loss.item())
            
            loss.backward() # backpropagation (вычисление градиентов)
            optimizer.step() # обновление весов сети
            optimizer.zero_grad() # обнуляем веса
            
            # вычислим accuracy на текущем train батче
            model_answers = torch.round(torch.sigmoid(logits))  # применяем сигмоиду и округляем
            train_accuracy = torch.sum(y_batch.cpu() == model_answers.cpu()) / len(y_batch)
            train_accuracies.append(train_accuracy.item())
            
            # вычислим ROC AUC на текущем train батче
            train_roc_auc = roc_auc_score(y_batch.detach().cpu().numpy(), model_answers.detach().cpu().numpy())
            train_roc_aucs.append(train_roc_auc)
            
            train_iterations.append((epoch * len(train_loader)) + i + 1)

            # выведем лосс, accuracy и ROC AUC на график раз в print_freq итераций обучения
            if (i+1) % 2000 == 0:
                fig, axes = plt.subplots(1, 3, figsize=(18, 5))
                
                axes[0].plot(train_iterations, train_losses)
                axes[0].set_title('Train losses')
                axes[0].set(xlabel='Iterations', ylabel='Loss')
                
                axes[1].plot(train_iterations, train_accuracies)
                axes[1].set_title('Train accuracies')
                axes[1].set(xlabel='Iterations', ylabel='Accuracy')
                
                axes[2].plot(train_iterations, train_roc_aucs)
                axes[2].set_title('Train ROC AUC')
                axes[2].set(xlabel='Iterations', ylabel='ROC AUC')
                
                plt.show()

                clear_output(wait=True)

        # после каждой эпохи получаем метрику качества на валидационной выборке
        model.train(False)
        val_accuracy, val_loss, val_roc_auc = evaluate(model, val_loader, loss_fn=loss_fn)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        val_roc_aucs.append(val_roc_auc)
        print("Эпоха {}/{}: val лосс, accuracy и ROC AUC:".format(epoch+1, n_epoch), val_loss, val_accuracy, val_roc_auc, end='\n')
        
    return model


In [ ]:
# снова объявим модель
model = create_model(models.resnet18(pretrained=True), 1)
model = model.to(device)

# выбираем функцию потерь
loss_fn = torch.nn.BCEWithLogitsLoss()

# выбираем алгоритм оптимизации и learning_rate. 
# вы можете экспериментировать с разными значениями learning_rate
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Запуск обучения модели

In [ ]:
# запустим обучение модели
# параметр n_epoch можно варьировать

start = timeit.default_timer()

model = train(model, loss_fn, optimizer, n_epoch=15)

end = timeit.default_timer()
execution_time = end - start

# Преобразуем время в часы, минуты и секунды
hours, rem = divmod(execution_time, 3600)
minutes, seconds = divmod(rem, 60)

print(f"Обучение длилось: {int(hours)} часов, {int(minutes)} минут, {seconds:.2f} секунд")

Эпоха 12/15: val лосс, accuracy и ROC AUC: 0.20167119707641473 0.95177823 0.9462669103464645
Epoch: 13


#### Прогоняем датасет test через сеть и получаем ответы

In [ ]:
def get_answer(model, dataloader):
    
    answers_arr = []
  
    images_pathes = [x[0] for x in dataloader.sampler.data_source.imgs]

    for i, batch in enumerate(dataloader):
        
        # получаем текущий батч
        X_batch, y_batch = batch
        
      
        
        # эта строка запрещает вычисление градиентов
        with torch.no_grad():
            # получаем ответы сети на картинки батча
            logits = model(X_batch.to(device))
            
            
            # вычисляем ответы сети
            y_pred = torch.sigmoid(logits) # применяем сигмоиду и округляем
            
            answers_arr.extend(y_pred.data.cpu().numpy())

            
            
    return answers_arr, images_pathes

In [ ]:
answers_arr, images_pathes = get_answer(model, test_loader)

#### Создание csv файла для отправки в kaggle

Проверка, преобразование к нежному виду и проверка массива с идентификаторами





In [ ]:
images_pathes

In [ ]:
images_names = [path.split('/')[-1].replace('.tif', '') for path in images_pathes]

In [ ]:
images_names

Проверка, преобразование и проверка массива с ответами

In [ ]:
answers_arr

In [ ]:
answers = [float(arr[0]) for arr in answers_arr]

answers

In [ ]:
import csv

# Проверка, что количество элементов в обоих массивах одинаково
if len(answers) != len(images_names):
    print("Количество элементов в массивах не совпадает.")
    exit()

# Создание CSV файла и запись данных
with open("output.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["id", "label"])  # Записываем заголовки столбцов
    for i in range(len(answers)):
        writer.writerow([images_names[i], answers[i]])  # Записываем данные

print("CSV файл успешно создан.")


In [ ]:
import pandas as pd

# Указываете путь к файлу CSV
csv_path = "convnet_output.csv"

# Чтение CSV файла и загрузка данных в DataFrame
data = pd.read_csv(csv_path)

# Выполнение операции вычитания 1 из столбца "label"
data["label"] = 1 - data["label"]

# Сохранение измененных данных обратно в CSV файл
data.to_csv(csv_path, index=False)

print("Столбец 'label' успешно изменен в файле convnet_output.csv.")


Сортировка csv файла. сначала будут идти изображения с раком, а потом без

In [ ]:
'''
import pandas as pd

# Проверка, что количество элементов в обоих массивах одинаково
if len(answers) != len(images_names):
    print("Количество элементов в массивах не совпадает.")
    exit()

# Создание DataFrame из данных массивов
data = pd.DataFrame({"id": images_names, "label": answers})

# Сортировка DataFrame по столбцу 'label' в порядке убывания
sorted_data = data.sort_values("label", ascending=False)

# Запись отсортированных данных в CSV файл
sorted_data.to_csv("output.csv", index=False)

print("CSV файл успешно создан и строки отсортированы.")
'''

CSV файл успешно создан и строки отсортированы.


In [ ]:
import pandas as pd

path = "output.csv"

data = pd.read_csv(path)

In [ ]:
data

,id,label
0,00006537328c33e284c973d7b39d340809f7271b,0.000396
1,0000ec92553fda4ce39889f9226ace43cae3364e,0.017440
2,00024a6dee61f12f7856b0fc6be20bc7a48ba3d2,0.028344
3,000253dfaa0be9d0d100283b22284ab2f6b643f6,0.997167
4,000270442cc15af719583a8172c87cd2bd9c7746,0.999118
...,...,...
57453,fffdd1cbb1ac0800f65309f344dd15e9331e1c53,0.890489
57454,fffdf4b82ba01f9cae88b9fa45be103344d9f6e3,0.990155
57455,fffec7da56b54258038b0d382b3d55010eceb9d7,0.997106
57456,ffff276d06a9e3fffc456f2a5a7a3fd1a2d322c6,0.851479
